In [ ]:
!pip install -q datasets
!pip install -q soundfile

In [22]:
import datasets
import numpy as np
import pandas as pd
import re
import string
import os
import subprocess
from tqdm.notebook import tqdm
import multiprocessing
import huggingface_hub

In [23]:
huggingface_hub.notebook_login()

In [5]:
!wget https://huggingface.co/datasets/allandclive/MakerereRadioSpeech_20Hrs/resolve/main/audio.zip
!wget https://huggingface.co/datasets/allandclive/MakerereRadioSpeech_20Hrs/raw/main/uncleaned.csv
!unzip -q audio.zip

In [6]:
df = pd.read_csv('uncleaned.csv')

In [7]:
def remove_multiple_spaces(text):
  return re.sub(r'\s+', ' ', text)

def sentence_case(input_string):
  if len(input_string) < 3:
    return input_string
  formatted_string = input_string[0].upper() + input_string[1:]
  if formatted_string[-1] not in string.punctuation:
    formatted_string += '.'
  return formatted_string

def remove_spaces_between_capitals(text):
    # Use regex to find uppercase letters with spaces in between and merge them
    return re.sub(r'(?<=[A-Z])\s+(?=[A-Z])', '', text)

def format_single_line(text):
  text = text.replace('A:', '')
  text = text.replace('B:', '')
  text = text.replace('[um]', '')
  text = text.replace('[part]', '')
  text = text.replace('[en]', '')
  text = text.strip()
  text = sentence_case(text)
  text = remove_spaces_between_capitals(text)
  return text

def format_lines(text):
  parts = text.split('\n')
  parts = [format_single_line(part) for part in parts]
  parts = [part for part in parts if len(part)>1]
  text = ' '.join(parts)
  text = remove_multiple_spaces(text)
  return text

def get_language(text):
  return 'eng' if '[en]' in text else 'lug'

df['cleaned_transcript'] = df['transcript'].apply(format_lines)

df['audio_language'] = df['transcript'].apply(get_language)

for i in range(2000, 2010):
  print(df['cleaned_transcript'][i])

Oku okugoberera. Okugoberera. Byemwakkiriziganyako. Mwakkiriziganyako. Webale nnyo.
Era oyina okubituukiriza. Ye sebo. Eyo y'enkwata y'ebb olwo tugamba nti oyo ebbanja alikwata buki?
N'ennaku z'omwezi bazitaddemu. n'omuwendo bagukoze ki? Bagutaddemu. N'emirundi emeka g'anakola ki? Gy'anazisasula. Gy'anasasula.
Ekyo kikulu nnyo. Ye sebo. Tugiyita kwegamba plan y'okusasula ebbanja.
Ogim. Ogitegeera era nga ogirina. Ye sebo. Kyetutera ogamba nti.
Ye sebo. Kubanga plan esasula kitegeeza nti omanyi omuwendo gw'esente gw'ogenda okusasula mu kiseera ekiddako.
Na ddi lw'ogenda ozisasula. Ye sebo. Tekigenda kugwa ko bugwi.
Ensonga endala nga okulemererwa okusasula.
Ensonga eguddewo. Ye sebo. Ebigwa tebiraze.
Kwata yo ebbanja. Ye sebo. Kuba bw'onooba onolikwata bulungi.


In [8]:
df[df.audio_language == 'lug']

,wav_filename,duration,transcript,cleaned_transcript,audio_language
0,audio_5fa142ebac4bd8.60517665.wav,8.01,A: Banyonyola bulungi mu lulimi lweba[part] lw...,Banyonyola bulungi mu lulimi lweba lwebategeer...,lug
1,audio_5fa142e82c8907.10197122.wav,4.38,A: Abataddemu ensimbi okulaba nti program eno ...,Abataddemu ensimbi okulaba nti program eno ekw...,lug
2,audio_5fa142f373f3a0.14347634.wav,5.37,A: Wamu n'ennima ey'omulembe ey'rmwanyi ebeera...,Wamu n'ennima ey'omulembe ey'rmwanyi ebeerawo ...,lug
3,audio_5fa142ef6fb398.44234122.wav,4.53,A: Ndi mulimisa\nB: [um] sebo\nA: Era nkola n'...,Ndi mulimisa. Sebo. Era nkola n'ekitongole eky...,lug
4,audio_5fa142e69bf891.02819656.wav,3.96,A: Oba munkola eno\nB: [um]\nA: Tukolagana n'e...,Oba munkola eno. Tukolagana n'ekitongole ekya ...,lug
...,...,...,...,...,...
11653,audio_5fbb5995c0b7a2.33101378.wav,8.97,A: Mubutuufu oblwadde webujja bujja kuba eyo [...,Mubutuufu oblwadde webujja bujja kuba eyo olab...,lug
11654,audio_5fbb5995a7aef9.81027427.wav,8.76,A: Okubeera nga ekiriisa kyanguwa okutuuka ku ...,Okubeera nga ekiriisa kyanguwa okutuuka ku mil...,lug
11655,audio_5fbb599764e022.13421389.wav,8.67,A: Waliwo abalimi abagamba nti waliwo akawuka ...,Waliwo abalimi abagamba nti waliwo akawuka aka...,lug
11656,audio_5fbb599834f5e4.44398828.wav,6.69,A: Kasobola okutanndika okwelolera okw'oyo mun...,Kasobola okutanndika okwelolera okw'oyo munnaf...,lug


In [37]:
!rm audio_folder/data/*.mp3

In [40]:
!mkdir -p audio_folder/data

def compress_audio(i):
  source_audio = f'audio/{df["wav_filename"][i]}'
  compressed_audio = f'audio_folder/data/{i:05}.mp3'

  # Omit any entries with short or missing transcripts
  if os.path.exists(source_audio) and len(df.cleaned_transcript[i]) > 5:
    try:
      subprocess.run(
          [
              'ffmpeg', '-hide_banner', '-loglevel', 'error',
              '-y', '-i', source_audio,
              compressed_audio
          ],
          check=True)
    except FileNotFoundError:
      print("ffmpeg not found. Please install ffmpeg.")
    except subprocess.CalledProcessError as e:
      print(f"Conversion failed with error: {e}")

with multiprocessing.Pool(processes=20) as pool:
  for _ in tqdm(pool.imap_unordered(
      compress_audio, df.index), total=len(df)):
    pass

  0%|          | 0/11658 [00:00<?, ?it/s]

In [41]:
metadata = pd.DataFrame()
metadata['file_name'] = [f'data/{i:05}.mp3' for i in df.index]
metadata['id'] = list(df.index)
metadata['text'] = list(df.cleaned_transcript)
metadata['audio_language'] = list(df.audio_language)

# Filter out the entries with no audio
metadata = metadata[metadata['file_name'].apply(
    lambda x: os.path.exists(os.path.join(f'audio_folder', x)))]

metadata.to_csv(f'audio_folder/metadata.csv', index=False)

In [42]:
dataset = datasets.load_dataset("audiofolder", data_dir=f"audio_folder")

Resolving data files:   0%|          | 0/10868 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [43]:
dataset.push_to_hub(
    'Sunbird/external-speech-data', config_name=f'makerere-radio-speech', private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/10867 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/109 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.95k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Sunbird/external-speech-data/commit/68aa393e2484b49bf164057c7a40977ae3ebb184', commit_message='Upload dataset', commit_description='', oid='68aa393e2484b49bf164057c7a40977ae3ebb184', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Sunbird/external-speech-data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Sunbird/external-speech-data'), pr_revision=None, pr_num=None)